In [16]:
import pandas as pd
import numpy as np

import cenpy as cen
from collections import OrderedDict

import warnings
warnings.simplefilter('ignore')

In [2]:
ny_zip_ref = pd.read_csv('New_York_State_Zip_Codes-County_FIPS_Cross-Reference.csv')

# only need to keep 'County Name' and 'ZIP Code'

ny_zip_ref = ny_zip_ref[['County Name', 'ZIP Code']]
ny_zip_ref

,County Name,ZIP Code
0,Albany,12007
1,Albany,12009
2,Albany,12023
3,Albany,12041
4,Albany,12045
...,...,...
2538,Yates,14561
2539,Yates,14837
2540,Yates,14842
2541,Yates,14857


In [3]:
# might not need this

sites = pd.read_csv('Voting_Poll_Sites.csv')
sites

,BOROUGH,SITE_STATUS,SITE_NAME,SITE_NUMBER,STREET_NUMBER,STREET_SUFFIX,STREET_NAME,POSTCODE,CITY,VOTER_ENTRANCE,HANDICAP_ENTRANCE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location
0,BROOKLYN,A,PS 12,11537,430,NaN,Howard Avenue,11233,Brooklyn,430 Howard Avenue (not used for voters),Enter on Prospect Place through school yard,NaN,NaN,16.0,41.0,363.0,3039174.0,3.014570e+09,Ocean Hill,NaN
1,NaN,A,Wyatt T. Walker Senior Housing,11517,2177,NaN,Frederick Douglass Boulevard,10026,New York,2177 Frederick Douglas Blvd,2177 Frederick Douglas Blvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,QUEENS,A,Allen AME Senior Center,10590,112-04,NaN,167 Street,11433,Jamaica,112-04 167th Street,112-04 167th Street,40.691478,-73.783335,12.0,27.0,266.0,4435291.0,4.123220e+09,South Jamaica,"(40.691478, -73.783335)"
3,BROOKLYN,A,Marlboro Memorial Post,11493,300,NaN,Avenue X,11223,Brooklyn,300 Avenue X,300 Avenue X,NaN,NaN,15.0,47.0,37402.0,3195077.0,3.071940e+09,Gravesend,NaN
4,BROOKLYN,A,PAL-Wynn Center,11629,495,NaN,Gates Avenue,11216,Brooklyn,495 Gates Avenue,495 Gates Avenue,NaN,NaN,3.0,36.0,265.0,3050974.0,3.018090e+09,Bedford,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,QUEENS,A,PS 107-Thomas A Dooley,Q0082,167-02,NaN,45 Avenue,11358,Flushing,167-02 45 Avenue,Side Entrance facing 168th Street near 45th Av...,40.755951,-73.799746,7.0,19.0,1195.0,4123454.0,4.054460e+09,East Flushing,"(40.755951, -73.799746)"
1227,MANHATTAN,A,Triangle House,M0015,112,NaN,East 128 Street,10035,New York,112 East 128 Street,112 East 128 Street,40.806665,-73.937085,11.0,9.0,242.0,1054499.0,1.017750e+09,East Harlem North,"(40.806665, -73.937085)"
1228,BROOKLYN,A,Warbasse Houses Bldg 5C,B0274,545,NaN,Neptune Avenue,11224,Brooklyn,545 Neptune Avenue,West Avenue,40.579987,-73.973001,13.0,47.0,354.0,3257102.0,3.072500e+09,West Brighton,"(40.579987, -73.973001)"
1229,BRONX,A,Pelham Pkwy Community Center,00036,785,NaN,Pelham Parkway North,10467,Bronx,785 Pelham Parkway North - around playgound,785 Pelham Parkway North - around playgound,40.857972,-73.864564,11.0,13.0,328.0,2093364.0,2.043490e+09,Bronxdale,"(40.857972, -73.864564)"


In [4]:
pres = pd.read_csv('2012President.csv')

# keep select columns
pres_cols = ["county", "office", "candidate", "party", "votes"]
pres = pres[pres_cols]

# keep only where office == President
pres = pres.loc[pres['office'] == "President"]
pres = pres.drop(['office'], axis = 1)

# get voting sums
agg_functions = {'county': 'first', 'candidate': 'first', 'party': 'first', 'votes': 'sum'}
pres_agg = pres.groupby(['county', 'candidate']).aggregate(agg_functions)

# drop index
pres_agg = pres_agg.reset_index(drop = True)
pres_agg

,county,candidate,party,votes
0,Albany,Barack Obama,DEM,175112.0
1,Albany,Gary Johnson,LBT,2722.0
2,Albany,Jill Stein,GRE,2476.0
3,Albany,Mitt Romney,REP,90128.0
4,Albany,Peta Lindsay,PSL,104.0
...,...,...,...,...
393,Yates,Jill Stein,GRN,134.0
394,Yates,Mitt Romney,REP,9596.0
395,Yates,Peta Lindsay,PSL,6.0
396,Yates,Virgil Goode,CST,34.0


In [14]:
counties = pres_agg.county.unique().tolist()
counties
# print(len(counties))

['Albany',
 'Allegany',
 'Bronx',
 'Broome',
 'Cattaraugus',
 'Cayuga',
 'Chautauqua',
 'Chenango',
 'Clinton',
 'Columbia',
 'Cortland',
 'Delaware',
 'Dutchess',
 'Erie',
 'Essex',
 'Franklin',
 'Fulton',
 'Genessee',
 'Greene',
 'Hamilton',
 'Herkimer',
 'Jefferson',
 'Lewis',
 'Livingston',
 'Madison',
 'Monroe',
 'Nassau',
 'New York',
 'Niagara',
 'Oneida',
 'Onondaga',
 'Ontario',
 'Orange',
 'Oswego',
 'Otsego',
 'Queens',
 'Rensselaer',
 'Richmond',
 'Rockland',
 'Saratoga',
 'Schenectady',
 'Schoharie',
 'Seneca',
 'St Lawrence',
 'Suffolk',
 'Sullivan',
 'Tioga',
 'Tompkins',
 'Ulster',
 'Warren',
 'Wayne',
 'Westchester',
 'Wyoming',
 'Yates']

In [6]:
# using cenpy get NY data
acs = cen.products.ACS()
acs.from_place('NY')

/Users/lmhartmann/miniconda3/lib/python3.9/site-packages/cenpy/products.py:791: UserWarning: Uncertain place identifier "NY". The place identifier should look something like "placename, state" or, for larger areas, like Combined Statistical Areas or Metropolitan Statistical Areas,"placename1-placename2, state1-state2-state3"
  geoms, variables, *rest = super(ACS, self).from_place(
/Users/lmhartmann/miniconda3/lib/python3.9/site-packages/cenpy/products.py:993: UserWarning: Cannot disambiguate placename NY. Picking the shortest, best matched placename, Onyx CDP, from Onyx CDP, Pony CDP
  warn(


Matched: NY to Onyx CDP within layer Census Designated Places


/var/folders/7f/sy5wyq412ng6bjc33vy79cg40000gn/T/ipykernel_8311/4206692168.py:3: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  acs.from_place('NY')


,geometry,state,county,tract,GEOID


In [7]:
# to see what categories we can look through
acs.tables

,description,columns
table_name,,
B01001,SEX BY AGE,"[B01001_001E, B01001_002E, B01001_003E, B01001..."
B01002,MEDIAN AGE BY SEX,"[B01002_001E, B01002_002E, B01002_003E]"
B01003,TOTAL POPULATION,[B01003_001E]
B02001,RACE,"[B02001_001E, B02001_002E, B02001_003E, B02001..."
B02008,WHITE ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02008_001E]
...,...,...
C27014,PUBLIC HEALTH INSURANCE BY WORK EXPERIENCE,"[C27014_001E, C27014_002E, C27014_003E, C27014..."
C27016,HEALTH INSURANCE COVERAGE STATUS BY RATIO OF I...,"[C27016_001E, C27016_002E, C27016_003E, C27016..."
C27017,PRIVATE HEALTH INSURANCE BY RATIO OF INCOME TO...,"[C27017_001E, C27017_002E, C27017_003E, C27017..."


## Create Dataframe on Sex Data

In [8]:
acs.filter_tables('SEX', by = 'description')

,description,columns
table_name,,
B01001,SEX BY AGE,"[B01001_001E, B01001_002E, B01001_003E, B01001..."
B01002,MEDIAN AGE BY SEX,"[B01002_001E, B01002_002E, B01002_003E]"
B05003,SEX BY AGE BY NATIVITY AND CITIZENSHIP STATUS,"[B05003_001E, B05003_002E, B05003_003E, B05003..."
B05004,MEDIAN AGE BY NATIVITY AND CITIZENSHIP STATUS ...,"[B05004_001E, B05004_002E, B05004_003E, B05004..."
B05008,SEX BY PLACE OF BIRTH BY YEAR OF ENTRY FOR THE...,"[B05008_001E, B05008_002E, B05008_003E, B05008..."
...,...,...
C27005,DIRECT-PURCHASE HEALTH INSURANCE BY SEX BY AGE,"[C27005_001E, C27005_002E, C27005_003E, C27005..."
C27006,MEDICARE COVERAGE BY SEX BY AGE,"[C27006_001E, C27006_002E, C27006_003E, C27006..."
C27007,MEDICAID/MEANS-TESTED PUBLIC COVERAGE BY SEX B...,"[C27007_001E, C27007_002E, C27007_003E, C27007..."


In [9]:
totalSexData = acs.filter_variables('B01001')

In [24]:
sexData0 = [] # 0 - 9
sexData1 = [] # 10 - 19
sexData2 = [] # 20 - 29
sexData3 = [] # 30 - 39
sexData4 = [] # 40 - 49
sexData5 = [] # 50 - 54

# the columns we are looking for
to_keep = ["Estimate!!Total:", "Estimate!!Total:!!Female:",
          "Estimate!!Total:!!Female:!!18 and 19 years", "Estimate!!Total:!!Female:!!20 to 24 years",
          "Estimate!!Total:!!Female:!!25 to 29 years", "Estimate!!Total:!!Female:!!30 to 34 years",
          "Estimate!!Total:!!Female:!!35 to 44 years", "Estimate!!Total:!!Female:!!45 to 54 years",
          "Estimate!!Total:!!Female:!!55 to 64 years", "Estimate!!Total:!!Female:!!65 to 74 years",
          "Estimate!!Total:!!Female:!!75 to 84 years", "Estimate!!Total:!!Female:!!85 years and over",
          "Estimate!!Total:!!Male:",
          "Estimate!!Total:!!Male:!!18 and 19 years", "Estimate!!Total:!!Male:!!20 to 24 years",
          "Estimate!!Total:!!Male:!!25 to 29 years", "Estimate!!Total:!!Male:!!30 to 34 years",
          "Estimate!!Total:!!Male:!!35 to 44 years", "Estimate!!Total:!!Male:!!45 to 54 years",
          "Estimate!!Total:!!Male:!!55 to 64 years", "Estimate!!Total:!!Male:!!65 to 74 years",
          "Estimate!!Total:!!Male:!!75 to 84 years", "Estimate!!Total:!!Male:!!85 years and over"]

# renamed column names
rename = ["Total:", "Total Female:",
          "Total Female: 18 and 19 years", "Total Female: 20 to 24 years",
          "Total Female: 25 to 29 years", "Total Female: 30 to 34 years",
          "Total Female: 35 to 44 years", "Total Female: 45 to 54 years",
          "Total Female: 55 to 64 years", "Total Female: 65 to 74 years",
          "Total Female: 75 to 84 years", "Total Female: 85 years and over",
          "Total Male:",
          "Total Male: 18 and 19 years", "Total Male: 20 to 24 years",
          "Total Male: 25 to 29 years", "Total Male: 30 to 34 years",
          "Total Male: 35 to 44 years", "Total Male: 45 to 54 years",
          "Total Male: 55 to 64 years", "Total Male: 65 to 74 years",
          "Total Male: 75 to 84 years", "Total Male: 85 years and over"]

# run in groups of 10 because of 'too many requests' error

for i in range(10):

    name = counties[i] + ", NY"
    currentDF = acs.from_county(name, variables = 'B01001')
    current = counties[i]
    
    # if first round through
    # create labels to rename the dataframes
    if i == 0:
        indexVals = totalSexData.index
        labels = totalSexData['label'].to_list()
        labelsDict = OrderedDict({})
        for key in indexVals:
            for value in labels:
                labelsDict[key] = value
                labels.remove(value)
                break

        labelsDict.update({"geometry": "geometry"})
        labelsDict.move_to_end("geometry", last = False)

        labelsDict.update({"GEOID": "GEOID"})
        labelsDict.move_to_end("GEOID", last = False)

        labelsDict.update({"state": "state"})
        labelsDict.update({"county": "county"})
        labelsDict.update({"tract": "tract"})
        labelsDict.update({"NAME": "NAME"})
    
    # rename the dataframe
    currentDF.columns = currentDF.columns.to_series().map(labelsDict)
    
    # drop the duplciate columns in the county dataframe
    currentDF = currentDF.loc[:, ~currentDF.columns.duplicated()].copy()
    
    # only keep the previously selected columns
    currentDF = currentDF[to_keep]

    # rename the columns 
    currentDF = currentDF.set_axis(rename, axis = 1)
    
    
    # get the sums for each category and put into a dictonary
    current_sums = currentDF.sum().to_dict()
    current_sums = OrderedDict(current_sums)

    current_sums.update({"County": current})
    current_sums.move_to_end("County", last = False)

    current_sums = dict(current_sums)

    # if first run through
    # get the headers for the final dataframe
    # append to the final dataframe
    if i == 0:
        headers = list(current_sums.keys())
        sexData.append(headers)
        
    # create inner_row to append
    # and append to final dataframe
    inner_row = list(current_sums.values())
    sexData0.append(inner_row)


# # make into dataframe
# sexDF = pd.DataFrame(sexData)

# # view
# sexDF
   

    

In [22]:
# write to sex data to csv
# as backup
sexDF.to_csv("NY_sexData.csv")

NameError: name 'sexDF' is not defined

In [ ]:
# albany = acs.from_county("Albany, NY", variables = 'B01001')

In [ ]:
# albany

In [ ]:
# indexVals = totalSexData.index
# labels = totalSexData['label'].to_list()

# from collections import OrderedDict

# labelsDict = OrderedDict({})
# for key in indexVals:
#     for value in labels:
#         labelsDict[key] = value
#         labels.remove(value)
#         break

# labelsDict.update({"geometry": "geometry"})
# labelsDict.move_to_end("geometry", last = False)
    
# labelsDict.update({"GEOID": "GEOID"})
# labelsDict.move_to_end("GEOID", last = False)

# labelsDict.update({"state": "state"})
# labelsDict.update({"county": "county"})
# labelsDict.update({"tract": "tract"})
# labelsDict.update({"NAME": "NAME"})

In [ ]:
# albany.columns = albany.columns.to_series().map(labelsDict)

In [ ]:
# albany = albany.loc[:, ~albany.columns.duplicated()].copy()
# albany

In [ ]:
# # keep specific columns
# to_keep = ["Estimate!!Total:", "Estimate!!Total:!!Female:",
#           "Estimate!!Total:!!Female:!!18 and 19 years", "Estimate!!Total:!!Female:!!20 to 24 years",
#           "Estimate!!Total:!!Female:!!25 to 29 years", "Estimate!!Total:!!Female:!!30 to 34 years",
#           "Estimate!!Total:!!Female:!!35 to 44 years", "Estimate!!Total:!!Female:!!45 to 54 years",
#           "Estimate!!Total:!!Female:!!55 to 64 years", "Estimate!!Total:!!Female:!!65 to 74 years",
#           "Estimate!!Total:!!Female:!!75 to 84 years", "Estimate!!Total:!!Female:!!85 years and over",
#           "Estimate!!Total:!!Male:",
#           "Estimate!!Total:!!Male:!!18 and 19 years", "Estimate!!Total:!!Male:!!20 to 24 years",
#           "Estimate!!Total:!!Male:!!25 to 29 years", "Estimate!!Total:!!Male:!!30 to 34 years",
#           "Estimate!!Total:!!Male:!!35 to 44 years", "Estimate!!Total:!!Male:!!45 to 54 years",
#           "Estimate!!Total:!!Male:!!55 to 64 years", "Estimate!!Total:!!Male:!!65 to 74 years",
#           "Estimate!!Total:!!Male:!!75 to 84 years", "Estimate!!Total:!!Male:!!85 years and over"]

# albany_select = albany[to_keep]
# albany_select

In [ ]:
# # rename nicely
# rename = ["Total:", "Total Female:",
#           "Total Female: 18 and 19 years", "Total Female: 20 to 24 years",
#           "Total Female: 25 to 29 years", "Total Female: 30 to 34 years",
#           "Total Female: 35 to 44 years", "Total Female: 45 to 54 years",
#           "Total Female: 55 to 64 years", "Total Female: 65 to 74 years",
#           "Total Female: 75 to 84 years", "Total Female: 85 years and over",
#           "Total Male:",
#           "Total Male: 18 and 19 years", "Total Male: 20 to 24 years",
#           "Total Male: 25 to 29 years", "Total Male: 30 to 34 years",
#           "Total Male: 35 to 44 years", "Total Male: 45 to 54 years",
#           "Total Male: 55 to 64 years", "Total Male: 65 to 74 years",
#           "Total Male: 75 to 84 years", "Total Male: 85 years and over"]

# albany_final = albany_select.set_axis(rename, axis = 1)
# albany_final

In [ ]:
# albany_sums = albany_final.sum().to_dict()
# albany_sums = OrderedDict(albany_sums)

# albany_sums.update({"County": "Albany"})
# albany_sums.move_to_end("County", last = False)

# albany_sums = dict(albany_sums)

# headers = list(albany_sums.keys())
# inner_row = list(albany_sums.values())

